<a href="https://colab.research.google.com/github/erha500/IMLO-Open-Assessment/blob/main/IMLO_Open_Assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms as T
from torchvision.transforms import v2

In [34]:
train_transform_augment = v2.Compose([
    v2.RandomResizedCrop([224,224], scale=[0.75,1.0], ratio=[1.0,1.0]),
    #v2.RandomPerspective(),
    v2.RandomHorizontalFlip(0.5),
    v2.RandomRotation(20),
    #v2.RandomVerticalFlip(0.5)
    #v2.RandomAutocontrast(),
    #v2.GaussianBlur(kernel_size=3),
    v2.ToTensor(),
    v2.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_transform = v2.Compose([
  v2.Resize([224,224]),
  v2.ToTensor(),
  v2.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


test_transform = v2.Compose([
  v2.Resize([224,224]),
  v2.ToTensor(),
  v2.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

C:\Users\erhan\AppData\Roaming\Python\Python311\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


Downloading Flowers102 dataset from datasets

In [35]:
training_data_augment = datasets.Flowers102(root="dataset", split="train", transform=train_transform_augment, download=True)

training_data = datasets.Flowers102(root="dataset", split="train", transform=train_transform, download=True)

val_data = datasets.Flowers102(root="dataset", split="val", transform=test_transform, download=True)

test_data = datasets.Flowers102(root="dataset", split="test", transform=test_transform, download=True)

In [36]:
batch_size = 64

#Create data loaders
train_dataloader = DataLoader(training_data_augment, batch_size = batch_size, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
  print(f"Shape of X [N, C, H, W]: {X.shape}")
  print(f"Shape of y: {y.shape} {y.dtype}")
  break

Shape of X [N, C, H, W]: torch.Size([64, 3, 224, 224])
Shape of y: torch.Size([64]) torch.int64


In [37]:
#Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

#Define model
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.model = nn.Sequential(
      #Convolutional layers
      nn.Conv2d(3, 32, kernel_size=1),
      nn.BatchNorm2d(32),
      nn.ReLU(),
      nn.Conv2d(32,32, kernel_size=1),
      nn.BatchNorm2d(32),
      nn.MaxPool2d(2, stride=2),

      nn.ReLU(),
      nn.Conv2d(32, 64, kernel_size=3, padding=1),
      nn.BatchNorm2d(64),
      nn.ReLU(),
      nn.Conv2d(64,64, kernel_size=3, padding=1),
      nn.BatchNorm2d(64),
      nn.ReLU(),
      nn.MaxPool2d(2, stride=2),

      nn.Conv2d(64, 128, kernel_size=3, padding=1),
      nn.BatchNorm2d(128),
      nn.ReLU(),
      nn.MaxPool2d(2, stride=2),
      nn.Conv2d(128,128, kernel_size=3, padding=1),
      nn.BatchNorm2d(128),
      nn.ReLU(),
      nn.MaxPool2d(2, stride=2),
      nn.AvgPool2d(2, stride=2),

      #Linear layers
      nn.Flatten(),
      nn.Linear(128 * 7 * 7, 1024),
      nn.BatchNorm1d(1024),
      nn.ReLU(),
      nn.Dropout(0.05),
      nn.Linear(1024, 512),
      nn.BatchNorm1d(512),
      nn.ReLU(),
      nn.Dropout(0.05),
      nn.Linear(512,256),
      nn.BatchNorm1d(256),
      nn.ReLU(),
      nn.Dropout(0.05),
      nn.Linear(256,102),
      nn.BatchNorm1d(102),
      nn.ReLU(),
      nn.Dropout(0.05),
      nn.LogSoftmax(dim=1)
    )

  def forward(self, x):
    return self.model(x)

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (model): Sequential(
    (0): Conv2d(3, 32, kernel_size=(1, 1), stride=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): ReLU()
    (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU()
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15)

In [38]:
#loss_fn = nn.CrossEntropyLoss()
loss_fn = nn.NLLLoss()
#optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.01) #weight decay is L2 regularisation
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.8)

In [39]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    #Compute prediction error
    pred = model(X)
    loss = loss_fn(pred, y)

    #Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 5 == 0:
      loss, current = loss.item(), (batch + 1) * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [40]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct) :>0.1f}%, Avg loss: {test_loss:>8f} \n")

    accuracy = 100 * correct
    return accuracy

In [41]:
epochs = 100

best_accuracy = 0
patience = 5
triggers = 0

for t in range(epochs):
  print(f"Epoch {t+1} \n ---------------------------------------")

  """#train on original data
  train_dataloader = DataLoader(training_data, batch_size=batch_size)
  train(train_dataloader, model, loss_fn, optimizer)"""

  #train on augmented data
  train(train_dataloader, model, loss_fn, optimizer)

  #testing on evaluation data
  accuracy = test(val_dataloader, model, loss_fn)

  if accuracy > best_accuracy:
    best_accuracy = accuracy
    triggers = 0
    torch.save(model.state_dict(), "model.pt")
    print("Saved model at current state")
  else:
    triggers += 1

  if triggers > patience:
    print("Stopping early")
    break

print("Testing -----------------------")
model.load_state_dict(torch.load("model.pt"))
model.eval()
test(test_dataloader, model, loss_fn)
print("Done!!!!")

Epoch 1 
 ---------------------------------------
loss: 4.891363 [   64/ 1020]
loss: 4.407516 [  384/ 1020]
loss: 4.370816 [  704/ 1020]
loss: 4.012832 [  960/ 1020]
Test Error: 
 Accuracy: 1.2%, Avg loss: 4.604004 

Saved model at current state
Epoch 2 
 ---------------------------------------
loss: 3.618779 [   64/ 1020]
loss: 3.897032 [  384/ 1020]
loss: 3.904202 [  704/ 1020]
loss: 4.017531 [  960/ 1020]
Test Error: 
 Accuracy: 11.4%, Avg loss: 3.935567 

Saved model at current state
Epoch 3 
 ---------------------------------------
loss: 3.573861 [   64/ 1020]
loss: 3.273135 [  384/ 1020]
loss: 3.336215 [  704/ 1020]
loss: 3.208517 [  960/ 1020]
Test Error: 
 Accuracy: 19.6%, Avg loss: 3.677950 

Saved model at current state
Epoch 4 
 ---------------------------------------
loss: 3.379833 [   64/ 1020]
loss: 2.950572 [  384/ 1020]
loss: 3.118651 [  704/ 1020]
loss: 3.281830 [  960/ 1020]
Test Error: 
 Accuracy: 22.2%, Avg loss: 3.535559 

Saved model at current state
Epoch 5 
 ---